In [8]:
import csv
from deap import gp

machine_alias = 'Hemera'
#filename = 'AAS Results/mapping-' + machine_alias + '.csv.txt' 

In [2]:
class Individual:
    def __init__(self, heuristic, fitness):
        temp_heuristic = heuristic
        if temp_heuristic[-1] == ',':
            temp_heuristic = temp_heuristic[:-1]
        if temp_heuristic[-1] == '"' or temp_heuristic[-1] == "'":
            temp_heuristic = temp_heuristic[:-1]
        if temp_heuristic[0] == "'" or temp_heuristic[0] == '"':
            temp_heuristc = temp_heuristic[1:]
        self.heuristic = temp_heuristic
        
        temp_fitness = fitness
        if temp_fitness[0] == "(":
            temp_fitness = temp_fitness[1:]
        if temp_fitness[-2:] == ",)":
            temp_fitness = temp_fitness[:-2]
        self.fitness = float(temp_fitness)
    
    def __lt__(self, ind):
        return self.fitness < ind.fitness
    
    def __eq__(self, ind):
        return self.fitness == ind.fitness
    
    def __str__(self):
        return self.heuristic + ' | ' + str(self.fitness)

In [3]:
def extractDictFromCSV(filename):
    data = []
    data_order = []
    data_dict_format = {}

    with open(filename, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
        for x in spamreader.next():
            temp = x
            if x != '':
                if x[0] == ' ':
                    temp = x[1:]
                data_order.append(temp)
                data_dict_format[temp] = None

        #print data_order

        for row in spamreader:
            data.append(dict(data_dict_format))
            count = len(data_dict_format.keys())
            #print "========="
            for x in row:
                temp = x
                if x != '':
                    if x[0] == ' ':
                        temp = x[1:]

                    #print data_order[len(data_dict_format.keys()) - count]
                    #print temp

                    data[-1][data_order[len(data_dict_format.keys())- count]] = temp
                    count -= 1
    return data

In [4]:
def extractIndividualsFromData(data, ind_field=['EndTree'], fitness_field=['Fitness'], condition=[]):
    population = []
    for d in data:
        flag = False
        if condition != []:
            for cond in condition:
                if not cond[0](cond[2](d[cond[1]]), cond[3]):
                    flag = True
                    break
        if flag:
            continue
        for k in range(0, len(ind_field)):
            ind_key = ind_field[k]
            fit_key = fitness_field[k]
            population.append(Individual(d[ind_key], d[fit_key]))
    
    return population

In [5]:
def topK(population, k):
    if k > len(population):
        return population
    return population[:k]

In [54]:
def heuristicDepthGE(ind, val):
    count = ind.count("IfThenElse")
    
    return count >= val

def heuristicDepthLE(ind, val):
    count = ind.count("IfThenElse")
    
    return count <= val

def heuristicDepthEQ(ind, val):
    count = ind.count("IfThenElse")
    
    return count == val

In [60]:
import operator

aas_filename = 'AAS Results/mapping-' + machine_alias + '.csv.txt' 
aas_fields = [['EndTree'], ['Fitness']]
aas_conditions = [[heuristicDepthEQ, 'EndTree', str, 2]]

gp_filename = 'GP Results/mapping-' + machine_alias + '.csv.txt'
gp_fields = [['Top1', 'Top2', 'Top3'], ['Fitness-top1', 'Fitness-top2', 'Fitness-top3']]
gp_conditions = [[operator.ge, 'Num Sims', int, 480000]]

alg = 'aas'

if alg == 'gp':
    filename = gp_filename
    fields = gp_fields
    conditions = gp_conditions
elif alg == 'aas':
    filename = aas_filename
    fields = aas_fields
    conditions = aas_conditions
else:
    filename = None
    fields = None
    conditions = None

In [61]:
csv_data = extractDictFromCSV(filename)
parsed_data = extractIndividualsFromData(csv_data, ind_field=fields[0], fitness_field=fields[1], condition=conditions)
parsed_data = sorted(parsed_data, reverse=True)

top = topK(parsed_data, 10)

#for d in top:
#    print d
for d in top:
    print '"' + d.heuristic + '",'

"IfThenElse(lowestCardGE(ARG0, 11), 'raise', IfThenElse(hasDoubles(ARG0), 'check', 'fold'))",
"IfThenElse(and_(lowestCardLE(ARG0, 9), and_(hasDoubles(ARG0), lowestCardGE(ARG0, 2))), 'check', IfThenElse(hasDoubles(ARG0), 'raise', 'fold'))",
"IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(hasDoubles(ARG0), 'check', 'fold'))",
"IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', 'raise'))",
"IfThenElse(and_(highestCardGE(ARG0, 10), hasDoubles(ARG0)), 'raise', IfThenElse(highestCardLE(ARG0, 13), 'fold', 'check'))",
"IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(and_(highestCardGE(ARG0, 2), lowestCardLE(ARG0, 11)), 'fold', 'raise'))",
"IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', 'check'))",
"IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotGE(ARG1, 27), 'check', 'fold'))",
"IfThenElse(lowestCardGE(ARG0, 10), 'raise', IfThenElse(totalPotLE(ARG1, 11), 'fold', 'check'))",
"IfThenElse(hasDoubles(ARG